In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hourly-energy-consumption/est_hourly.paruqet
/kaggle/input/hourly-energy-consumption/DOM_hourly.csv
/kaggle/input/hourly-energy-consumption/EKPC_hourly.csv
/kaggle/input/hourly-energy-consumption/DUQ_hourly.csv
/kaggle/input/hourly-energy-consumption/DAYTON_hourly.csv
/kaggle/input/hourly-energy-consumption/PJME_hourly.csv
/kaggle/input/hourly-energy-consumption/PJM_Load_hourly.csv
/kaggle/input/hourly-energy-consumption/NI_hourly.csv
/kaggle/input/hourly-energy-consumption/FE_hourly.csv
/kaggle/input/hourly-energy-consumption/COMED_hourly.csv
/kaggle/input/hourly-energy-consumption/AEP_hourly.csv
/kaggle/input/hourly-energy-consumption/pjm_hourly_est.csv
/kaggle/input/hourly-energy-consumption/DEOK_hourly.csv
/kaggle/input/hourly-energy-consumption/PJMW_hourly.csv


In [64]:
df = pd.read_csv('../input/hourly-energy-consumption/PJME_hourly.csv')
df.head()

,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0


In [65]:
df = df.set_index("Datetime")
df = df.sort_index()

In [66]:
df.index = pd.to_datetime(df.index)

In [67]:
df.head()

,PJME_MW
Datetime,
2002-01-01 01:00:00,30393.0
2002-01-01 02:00:00,29265.0
2002-01-01 03:00:00,28357.0
2002-01-01 04:00:00,27899.0
2002-01-01 05:00:00,28057.0


In [69]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

In [70]:
df_train = df.loc['2002-01-01':'2014-12-31']

In [71]:
df.shape

(145366, 1)

In [72]:
df_train.shape

(113926, 1)

In [73]:
df_test = df.drop(df_train.index)

In [74]:
df_test.shape

(31440, 1)

In [75]:
kfold = KFold(n_splits=5, shuffle=True, random_state=2)

In [76]:
def cross_val(X, y, model):
    scores = cross_val_score(model, X, y, scoring='neg_root_mean_squared_error', cv=kfold)
    rmse = (-scores.mean())
    return rmse

In [77]:
def create_columns(df):
    df = df.copy()
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofweek'] = df.index.dayofweek
    df['daypfyear'] = df.index.dayofyear
    df['quarter'] = df.index.quarter
    df['hour'] = df.index.hour

    return df

In [78]:
df_train = create_columns(df_train)
y_train = df_train['PJME_MW']
X_train = df_train.drop('PJME_MW', axis=1)

df_test = create_columns(df_test)
y_test = df_test['PJME_MW']
X_test = df_test.drop('PJME_MW', axis=1)

In [79]:
xgb_1 = XGBRegressor(booster='gbtree', learning_rate=0.1, max_depth=3, n_estimators=100)

In [80]:
cross_val(X_train, y_train, xgb_1)

3049.779626100818

In [81]:
def grid_search(params, reg=XGBRegressor(booster='gbtree')):
    grid_reg = GridSearchCV(reg, params, scoring='neg_mean_squared_error', cv=kfold)
    grid_reg.fit(X_train, y_train)
    best_params = grid_reg.best_params_
    print("Best params:", best_params)
    best_score = np.sqrt(-grid_reg.best_score_)
    print("Best score:", best_score)

In [82]:
grid_search(params={'max_depth': [1,2,3,4,5], 'n_estimators':[50, 100, 150, 175, 200]})

Best params: {'max_depth': 5, 'n_estimators': 200}
Best score: 1754.0122884904827


Best score so far is 401

In [ ]:
grid_search(params={'max_depth': [8], 'n_estimators': [2000, 4000, 6000, 8000]})

In [ ]:
# Try out these hyperparameters

'gamma': [0.05, 0.1, 0.5, 1]
'colsample_bynode': [0.5, 0.6, 0.7, 0.8, 0.9, 1], 
'colsample_bylevel': [0.5, 0.6, 0.7, 0.8, 0.9, 1], 
'colsample_bytree':[0.5, 0.6, 0.7, 0.8, 0.9, 1]
'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4],
'max_depth': [8], 
'n_estimators': [200, 400, 600, 800]
'min_child_weight':[1, 2, 3, 4, 5]

In [ ]:
#Best ones are the following:
'gamma': [0]
'colsample_bynode': [1], 
'colsample_bylevel': [1], 
'colsample_bytree':[1]
'learning_rate': [0.01],
'max_depth': [3], 
'n_estimators': [500]
'min_child_weight':[1]

In [ ]:
# Future steps is to make a new xgboost model with the hyperparameters, run and fit it, get the model and predict the test data to see if it matches.